# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
tf.__version__

'2.6.0'

### Importar o dataset

In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head(50)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [7]:
dataset.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [8]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [9]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X[:, 2])

[0 0 0 ... 0 1 0]


In [10]:
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
np.shape(X_train)

(8000, 12)

In [14]:
X_train

array([[-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
        -1.03227043,  0.32472465]])

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [15]:
tf.random.set_seed(0) #Define que os casos de teste e os casos para praticar serão sempre iguais

In [16]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the second hidden layer

In [18]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the output layer

In [19]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compiling the ANN

In [20]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Part 3 - Training the ANN

### Training the ANN on the Training set

In [21]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 878us/step - loss: 0.6061 - accuracy: 0.6874
Epoch 2/100
250/250 [==============================] - 0s 804us/step - loss: 0.4607 - accuracy: 0.8046
Epoch 3/100
250/250 [==============================] - 0s 854us/step - loss: 0.4288 - accuracy: 0.8116
Epoch 4/100
250/250 [==============================] - 0s 819us/step - loss: 0.4199 - accuracy: 0.8165
Epoch 5/100
250/250 [==============================] - 0s 815us/step - loss: 0.4136 - accuracy: 0.8223
Epoch 6/100
250/250 [==============================] - 0s 803us/step - loss: 0.4076 - accuracy: 0.8250
Epoch 7/100
250/250 [==============================] - 0s 835us/step - loss: 0.4009 - accuracy: 0.8296
Epoch 8/100
250/250 [==============================] - 0s 920us/step - loss: 0.3939 - accuracy: 0.8319
Epoch 9/100
250/250 [==============================] - 0s 839us/step - loss: 0.3869 - accuracy: 0.8355
Epoch 10/100
250/250 [==============================] - 0s 803us/step - l

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [22]:
y_pred = ann.predict(X_test)
y_pred

array([[0.28365082],
       [0.16923717],
       [0.10380057],
       ...,
       [0.11263412],
       [0.174887  ],
       [0.24651918]], dtype=float32)

In [23]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1527   68]
 [ 202  203]]


In [25]:
#cm[0, 0]: Previsão correta op1
#cm[0, 1]: Previsão incorreta op1
#cm[1, 0]: Previsão incorreta op2
#cm[1, 1]: Previsão correta op2

# Funções de ativação experimentadas: relu, softmax, tanh

#Acurácia das combinações
# relu + relu: 0,86
# relu + softmax: 0,8585
# relu + tanh: 0,8605

# softmax + relu: 0,8635
# softmax + softmax: 0,8575
# softmax + tanh: 0,859

# tanh + relu: 0,8605
# tanh + softmax: 0,8605
# tanh + tanh: 0,865

from sklearn.metrics import (accuracy_score)
accuracy_score(y_test, y_pred)

0.865

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)

Modificado por
> Álvaro Leles Guimarães